In [2]:
!pip install torchsummary

In [22]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms,models
from torch.utils.data import DataLoader
from torchsummary import summary


In [18]:
# Define the data transformations (normalization and data augmentation)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 (VGG input size)
    transforms.ToTensor(),          # Convert image to PyTorch Tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize the images
])

# Load CIFAR-10 dataset
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Define data loaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [19]:


class VGG19(nn.Module):
    def __init__(self, num_classes=1000):
        super(VGG19, self).__init__()

        # Feature extraction layers: Convolutional and pooling layers
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),  # 3 input channels, 64 output channels, 3x3 kernel, 1 padding
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),  # Max pooling with 2x2 kernel and stride 2

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        # Pooling Layer
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(7, 7))

        # Fully connected layers for classification
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),  # 512 channels, 7x7 spatial dimensions after max pooling
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout layer with 0.5 dropout probability
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, num_classes),  # Output layer with 'num_classes' output units
        )

    def forward(self, x):
        x = self.feature_extractor(x)  # Pass input through the feature extractor layers
        x= self.avgpool(x)
        x = x.view(x.size(0), -1)  # Flatten the output for the fully connected layers
        x = self.classifier(x)  # Pass flattened output through the classifier layers
        return x



In [20]:
# Instantiate the model
model = VGG19(num_classes=10)  # CIFAR-10 has 10 classes
model = model.cuda()  # Move model to GPU if available

# Print the model summary
summary(model, (3, 224, 224))

# Define the loss function (cross-entropy loss for classification)
criterion = nn.CrossEntropyLoss()

# Define the optimizer (Adam or SGD)
optimizer = optim.Adam(model.parameters(), lr=0.001)  # You can also use optim.SGD


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [23]:
# Load pretrained VGG19 model
model = models.vgg19(pretrained=True)

# Modify the classifier for your dataset (e.g., 10 classes for CIFAR-10)
model.classifier[6] = nn.Linear(4096, 10)

# Freeze all layers except the classifier
for param in model.features.parameters():
    param.requires_grad = False

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:06<00:00, 84.2MB/s]


In [25]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for batch_idx,(inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if (batch_idx + 1) % 10 == 0:  # Print every 10 batches
            print(f"Epoch [{epoch + 1}/{num_epochs}], Batch [{batch_idx + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

print("Finished Training")


Epoch [1/10], Batch [10/782], Loss: 0.8863
Epoch [1/10], Batch [20/782], Loss: 0.6470
Epoch [1/10], Batch [30/782], Loss: 0.7139
Epoch [1/10], Batch [40/782], Loss: 0.7459
Epoch [1/10], Batch [50/782], Loss: 0.8861
Epoch [1/10], Batch [60/782], Loss: 0.6194
Epoch [1/10], Batch [70/782], Loss: 1.1164
Epoch [1/10], Batch [80/782], Loss: 0.6780
Epoch [1/10], Batch [90/782], Loss: 0.9033
Epoch [1/10], Batch [100/782], Loss: 0.8752
Epoch [1/10], Batch [110/782], Loss: 0.6276
Epoch [1/10], Batch [120/782], Loss: 0.8578
Epoch [1/10], Batch [130/782], Loss: 0.9184
Epoch [1/10], Batch [140/782], Loss: 1.1707
Epoch [1/10], Batch [150/782], Loss: 0.7914
Epoch [1/10], Batch [160/782], Loss: 0.7372
Epoch [1/10], Batch [170/782], Loss: 0.4947
Epoch [1/10], Batch [180/782], Loss: 0.7922
Epoch [1/10], Batch [190/782], Loss: 0.8658
Epoch [1/10], Batch [200/782], Loss: 0.5676
Epoch [1/10], Batch [210/782], Loss: 0.9345
Epoch [1/10], Batch [220/782], Loss: 0.6746
Epoch [1/10], Batch [230/782], Loss: 0.64

In [26]:
# Testing the model
model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # No need to track gradients during testing
    for inputs, labels in test_loader:
        inputs, labels = inputs.cuda(), labels.cuda()  # Move data to GPU if available
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)  # Get the class with the highest score
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print accuracy
accuracy = 100 * correct / total
print(f'Accuracy on the test set: {accuracy:.2f}%')


Accuracy on the test set: 85.15%
